In [3]:
import pandas as pd
import numpy as np

import sqlite3

In [6]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection successfull")
    except Error as e:
        print(f"The error '{e}' occured")
    return connection

In [7]:
engine = create_connection("SQL_14_07.db")

Connection successfull


In [8]:
df = pd.read_csv('GlobalWeatherRepository.csv')

In [9]:
df.to_sql('weather',engine,if_exists='replace',index=False)

12379

In [15]:
query = '''SELECT * FROM weather limit 10'''
db = pd.read_sql_query(sql = query, con = engine)
db

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1715849100,2024-05-16 13:15,26.6,79.8,Partly Cloudy,...,8.4,26.6,1,1,04:50 AM,06:50 PM,12:12 PM,01:11 AM,Waxing Gibbous,55
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1715849100,2024-05-16 10:45,19.0,66.2,Partly cloudy,...,1.1,2.0,1,1,05:21 AM,07:54 PM,12:58 PM,02:14 AM,Waxing Gibbous,55
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1715849100,2024-05-16 09:45,23.0,73.4,Sunny,...,10.4,18.4,1,1,05:40 AM,07:50 PM,01:15 PM,02:14 AM,Waxing Gibbous,55
3,Andorra,Andorra La Vella,42.50,1.52,Europe/Andorra,1715849100,2024-05-16 10:45,6.3,43.3,Light drizzle,...,0.7,0.9,1,1,06:31 AM,09:11 PM,02:12 PM,03:31 AM,Waxing Gibbous,55
4,Angola,Luanda,-8.84,13.23,Africa/Luanda,1715849100,2024-05-16 09:45,26.0,78.8,Partly cloudy,...,183.4,262.3,5,10,06:12 AM,05:55 PM,01:17 PM,12:38 AM,Waxing Gibbous,55
5,Antigua and Barbuda,Saint John's,17.12,-61.85,America/Antigua,1715849100,2024-05-16 04:45,26.0,78.8,Partly cloudy,...,1.2,4.5,1,1,05:36 AM,06:32 PM,01:05 PM,01:14 AM,Waxing Gibbous,55
6,Argentina,Buenos Aires,-34.59,-58.67,America/Argentina/Buenos_Aires,1715849100,2024-05-16 05:45,8.0,46.4,Clear,...,4.0,5.3,1,1,07:43 AM,05:59 PM,02:36 PM,01:04 AM,Waxing Gibbous,55
7,Armenia,Yerevan,40.18,44.51,Asia/Yerevan,1715849100,2024-05-16 12:45,19.0,66.2,Partly cloudy,...,0.8,0.9,1,1,05:45 AM,08:12 PM,01:17 PM,02:31 AM,Waxing Gibbous,55
8,Australia,Canberra,-35.28,149.22,Australia/Sydney,1715849100,2024-05-16 18:45,9.0,48.2,Clear,...,3.7,5.4,1,1,06:52 AM,05:07 PM,01:31 PM,No moonset,Waxing Gibbous,55
9,Austria,Vienna,48.20,16.37,Europe/Vienna,1715849100,2024-05-16 10:45,16.0,60.8,Partly cloudy,...,3.7,4.4,1,1,05:14 AM,08:29 PM,01:00 PM,02:42 AM,Waxing Gibbous,55


Максимальное значение температуры среди средних по странам

In [14]:
query = '''
with avg_temp_by_country as 
(SELECT 
    country,
    AVG(temperature_celsius) as avg_temp
FROM weather
GROUP BY country)

SELECT
    country,
    MAX(avg_temp) as max_temp
FROM avg_temp_by_country
'''
db = pd.read_sql_query(sql = query, con = engine)
db

,country,max_temp
0,Saudi Arabien,45.0


Страны, города которых не попадают в топ-10 по наиболее высокой температуре

In [19]:
query = '''
SELECT 
    country,
    location_name,
    AVG(temperature_celsius) as avg_temp
FROM weather
GROUP BY location_name
ORDER BY avg_temp desc
limit 10
'''
db = pd.read_sql_query(sql = query, con = engine)
db

,country,location_name,avg_temp
0,Saudi Arabien,Ar Riyadh,45.000000
1,Kuwait,Kuwait,44.400000
2,Kuwait,Kuwait City,42.857143
3,Iraq,Baghdad,42.781250
4,Saudi Arabia,Riyadh,42.717460
5,Sudan,Khartoum,40.312698
6,Marrocos,Morocco City,40.300000
7,Qatar,Doha,39.020313
8,Turkménistan,Krasnyy Turkmenistan,37.800000
9,Chad,N'djamena,37.517187


In [18]:
query = '''
with top_10_cities as 
(SELECT 
    country,
    location_name,
    AVG(temperature_celsius) as avg_temp
FROM weather
GROUP BY location_name
ORDER BY avg_temp desc
limit 10)

select
    distinct(country)
from weather
    where country not in (select distinct(country) from top_10_cities)
'''
db = pd.read_sql_query(sql = query, con = engine)
db

,country
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
196,Saint-Vincent-et-les-Grenadines
197,Südkorea
198,Bélgica
199,火鸡


Для каждой из стран вычислите разницу средней температуры по стране и средней температуры по всем странам. Помимо этого, вычислите на сколько процентов средняя температура по стране отличается от средней температуры по всем странам.

In [21]:
query = ''' 
SELECT 
    AVG(temperature_celsius) as avg_temp 
FROM weather

'''
db = pd.read_sql_query(sql = query, con = engine)
db

,avg_temp
0,26.185685


In [32]:
query = '''
WITH avg_temp_all as 
(SELECT 
    AVG(temperature_celsius) as avg_temp 
FROM weather),
avg_by_country as 
(select
    DISTINCT(country),
    AVG(temperature_celsius) OVER(PARTITION BY country) as avg_temp
from weather)

SELECT 
    abc.country,
    abc.avg_temp as avg_by_country,
    ata.avg_temp as avg_all,
    (abc.avg_temp - ata.avg_temp) as diff_abs,
    ((abc.avg_temp - ata.avg_temp)/ata.avg_temp)*100 as diff_perc
from avg_temp_all ata, avg_by_country abc
'''
db = pd.read_sql_query(sql = query, con = engine)
db

,country,avg_by_country,avg_all,diff_abs,diff_perc
0,Afghanistan,25.392187,26.185685,-0.793498,-3.030274
1,Albania,30.234375,26.185685,4.048690,15.461461
2,Algeria,27.882812,26.185685,1.697127,6.481125
3,Andorra,17.253125,26.185685,-8.932560,-34.112380
4,Angola,26.826563,26.185685,0.640877,2.447433
...,...,...,...,...,...
205,Гватемала,15.900000,26.185685,-10.285685,-39.279802
206,Польша,24.200000,26.185685,-1.985685,-7.583095
207,Турция,34.000000,26.185685,7.814315,29.841932
208,كولومبيا,23.100000,26.185685,-3.085685,-11.783864


In [35]:
query = '''
WITH temp_country as (
SELECT country,
    avg(temperature_celsius) OVER(partition by country) as avg_t_country,
    avg(temperature_celsius) OVER() as avg_t
FROM weather)

SELECT distinct country, avg_t_country, avg_t, avg_t_country - avg_t as  diff, (avg_t_country - avg_t)*100/avg_t as rel
FROM temp_country
'''
db = pd.read_sql_query(sql = query, con = engine)
db

,country,avg_t_country,avg_t,diff,rel
0,Afghanistan,25.392187,26.185685,-0.793498,-3.030274
1,Albania,30.234375,26.185685,4.048690,15.461461
2,Algeria,27.882812,26.185685,1.697127,6.481125
3,Andorra,17.253125,26.185685,-8.932560,-34.112380
4,Angola,26.826563,26.185685,0.640877,2.447433
...,...,...,...,...,...
205,Гватемала,15.900000,26.185685,-10.285685,-39.279802
206,Польша,24.200000,26.185685,-1.985685,-7.583095
207,Турция,34.000000,26.185685,7.814315,29.841932
208,كولومبيا,23.100000,26.185685,-3.085685,-11.783864
